<a href="https://colab.research.google.com/github/Varun-Mulchandani/591-RTDL/blob/main/Attn_Batch_GRU%2BNew_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random 

from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.nn.utils.prune as prune 

import nltk
from nltk.translate.bleu_score import corpus_bleu

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# !pip install sacremoses

# ! pip install transformers
# from transformers import BertTokenizer
# from transformers import FlaubertTokenizer

# tokenizer_eng = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case = True)
# tokenizer_eng = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
# tokenizer_fre = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased', do_lower_case = True)

if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [ ]:
%%capture

! pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util

In [ ]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

MAX_LENGTH = 10

class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: 'PAD', 1:'SOS', 2:'EOS'}
    self.n_words = 2

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      #Creating our own one hot vector
      self.word2index[word] = self.n_words
      #Creating the count of words
      self.word2count[word] = 1
      # creating indices for each word in a dictionary
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

#Cleaning the data by normalising and converting to a uniformly Ascii format
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
  )

def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  s = re.sub(r'([.!?])', r' \1', s)
  s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
  return s

def readLangs(lang1, lang2, reverse = False):
  print('Reading lines...')

  lines = open('/content/drive/MyDrive/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

  pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

  if reverse:
    pairs = [list(reversed(p)) for p in pairs]
    inp_lang = Lang(lang2)
    out_lang = Lang(lang1)
  else:
    inp_lang = Lang(lang1)
    outp_lang = Lang(lang2)
  
  return inp_lang, out_lang, pairs

def filterPair(p):
  return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def prepareData(lang1, lang2, reverse=False):
  input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
  print('before', len(pairs))
  pairs = filterPairs(pairs)
  print('after', len(pairs))
  for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
  return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng','fra',True)
# print(random.sample(pairs, 1))
# print(pairs[0])
print(len(pairs))
print(len(pairs[0]))
random.Random(0).shuffle(pairs)
print(len(pairs))
print(len(pairs[0]))

Reading lines...
before 135842
after 10599
10599
2
10599
2


In [ ]:
print(pairs[:10])

[['je suis a boston .', 'i m in boston .'], ['je ne suis pas une enfant .', 'i m not a child .'], ['j emploie un bol et une cuillere .', 'i m using a bowl and a spoon .'], ['je ne suis pas du matin .', 'i m not a morning person .'], ['je me mets a t apprecier .', 'i m starting to like you .'], ['vous etes la plus mignonne .', 'you re the sweetest .'], ['je suis en retard sur le programme .', 'i m behind schedule .'], ['tu es incroyable .', 'you re amazing .'], ['nous sommes des adultes .', 'we re adults .'], ['je suis fatiguee de faire la file .', 'i m tired of waiting in line .']]


In [ ]:
test_pairs = pairs[8000:]
pairs = pairs[:8000]

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
print(len(pairs))

8000


In [ ]:
for i in range(10599, -1, -1):
  if not (i % 32):
    print(i)
    break

10592


In [ ]:
# Pad sequence does a transpose all by itself

a = torch.tensor([1, 2, 3])
b = torch.tensor([5, 6])
c = torch.tensor([1, 3, 6, 8])
torch.nn.utils.rnn.pad_sequence([a, b, c])

tensor([[1, 5, 1],
        [2, 6, 3],
        [3, 0, 6],
        [0, 0, 8]])

In [ ]:
batch_size = 32

i = 0

batches = []

while i < 8000:
  j = 0
  train_french = []
  train_english = []
  train_french_lengths = []
  train_english_lengths = []
  while j < 32:
    sf, se = pairs[i]
    sf_tensor = tensorFromSentence(input_lang, sf)
    se_tensor = tensorFromSentence(output_lang, se)
    train_french_lengths.append(len(sf_tensor))
    train_english_lengths.append(len(se_tensor))
    train_french.append(sf_tensor)
    train_english.append(se_tensor)
    j += 1
    i += 1
  train_english = torch.nn.utils.rnn.pad_sequence(train_english)
  train_french = torch.nn.utils.rnn.pad_sequence(train_french)
  batches.append((train_french, train_english, train_french_lengths, train_english_lengths))

Creating batches of the train dataset

In [ ]:
class encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.rnn = nn.GRU(self.hidden_size, self.hidden_size, num_layers = 2)
  def forward(self, input, hidden, batch_size):
    embedded = self.embedding(input).view(1, batch_size, -1)
    output = embedded
    output, hidden = self.rnn(output, hidden)
    return output, hidden
  def init_hidden(self, batch_size):
    return torch.zeros(2, batch_size, self.hidden_size, device = device)

In [ ]:
# encoder1 = encoder(input_lang.n_words, 256)

# input_tensor = batches[0][0]
# encoder_hidden = encoder1.init_hidden(32)
# input_length = input_tensor.size()[0]

# encoder_outputs = torch.zeros(input_length, 32, encoder1.hidden_size)

# for ei in range(input_length):
#   encoder_output, encoder_hidden = encoder1(
#       input_tensor[ei], encoder_hidden, 32)
#   encoder_outputs[ei] = encoder_output[0]

In [ ]:
# class decoder(nn.Module):
#   def __init__(self, hidden_size, output_size, dropout_p = 0.1):
#     super(decoder, self).__init__()
#     self.hidden_size = hidden_size
#     self.output_size = output_size
#     self.rnn = nn.GRU(self.hidden_size, self.hidden_size, num_layers = 2)
#     self.out = nn.Linear(self.hidden_size, self.output_size)
#     self.dropout = nn.Dropout(dropout_p)
#     self.embedding = nn.Embedding(self.output_size, self.hidden_size)

#   def forward(self, input, hidden, batch_size):
#     embedded = self.embedding(input).view(1, batch_size, -1)
#     output = F.relu(embedded)
#     output, hidden = self.rnn(output, hidden)
#     output = F.log_softmax(self.out(output[0]), dim = 1)
#     return output, hidden
#   def init_hidden(self, batch_size):
#     return torch.zeros(2, batch_size, self.hidden_size, device = device)

In [ ]:
class Attn_Decoder(nn.Module):
  def __init__(self, hidden_size, output_size, dropout_p = 0.1):
    super(Attn_Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout_p = dropout_p

    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)
    self.rnn = nn.GRU(self.hidden_size, self.hidden_size, num_layers = 2)
    self.out = nn.Linear(self.hidden_size, self.output_size)

  def forward(self, input, hidden, encoder_outputs, batch_size, tar_len):
    embedded = self.embedding(input).view(1, batch_size, -1)
    embedded = self.dropout(embedded)
    # attn = nn.Linear(self.hidden_size * 2, max(tar_len, encoder_outputs.size()[0])).to(device)
    attn = nn.Linear(self.hidden_size * 2, encoder_outputs.size()[0]).to(device)
    attn_weights = F.softmax(
        attn(torch.cat((embedded[0], hidden[0]), 1)), dim = 1
    )
    encoder_output_length = encoder_outputs.size()[0]
    encoder_outputs = encoder_outputs.view(batch_size, encoder_output_length, self.hidden_size)
    attn_applied = []

    for i in range(batch_size):
      enc_outputs_curr = encoder_outputs[0]
      attn_weights_curr = attn_weights[0]
      attn_applied_curr = torch.bmm(attn_weights_curr.unsqueeze(0).unsqueeze(0), enc_outputs_curr.unsqueeze(0))
      attn_applied.append(attn_applied_curr)

    attn_applied = torch.cat(attn_applied, dim = 0)
    attn_applied = attn_applied.view(-1, batch_size, self.hidden_size)
    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)
    output = F.relu(output)
    output, hidden = self.rnn(output, hidden)

    output = F.log_softmax(self.out(output[0]), dim = 1)
    return output, hidden

  def initHidden(self, batch_size):
    return torch.zeros(2, batch_size, self.hidden_size, device = device)

In [ ]:
# decoder1 = Attn_Decoder(256, output_lang.n_words)

# target_tensor = batches[0][1]
# target_length = target_tensor.size()[0]
# decoder_hidden = encoder_hidden
# decoder_input = torch.tensor([SOS_token] * 32)
# for di in range(target_length):
#   decoder_output, decoder_hidden, decoder_attention = decoder1(decoder_input, decoder_hidden, encoder_outputs, 32, target_length)
#   # print(decoder_input)
#   topv, topi = decoder_output.data.topk(1)
#   # print(topi.size())
#   decoder_input = topi.view(1, -1)[0]
#   # print(decoder_input.size())

In [ ]:
training_required = False

if training_required:
  hidden_size = 256
  torch.manual_seed(0)
  torch.cuda.manual_seed(0)
  encoder1 = encoder(input_lang.n_words, hidden_size).to(device)
  torch.manual_seed(0)
  torch.cuda.manual_seed(0)
  decoder1 = Attn_Decoder(hidden_size, output_lang.n_words).to(device)
  epochs = 200
  criterion = nn.NLLLoss()
  encoder_optimizer = optim.SGD(encoder1.parameters(), lr = 0.01)
  decoder_optimizer = optim.SGD(decoder1.parameters(), lr = 0.01)

  enc_sched = torch.optim.lr_scheduler.StepLR(encoder_optimizer, 32, gamma = 0.9999)
  dec_sched = torch.optim.lr_scheduler.StepLR(decoder_optimizer, 32, gamma = 0.9999)
  teacher_forcing_ratio = 0.5

  for epoch in range(epochs):
    epoch_loss = 0
    for batch in batches:

      loss = 0 
      input_tensor = batch[0]
      target_tensor = batch[1]

      input_length = input_tensor.size()[0]
      target_length = target_tensor.size()[0]

      encoder_optimizer.zero_grad()
      decoder_optimizer.zero_grad()

      encoder_hidden = encoder1.init_hidden(32)

      encoder_outputs = torch.zeros(input_length, 32, encoder1.hidden_size).to(device)
      for ei in range(input_length):
        encoder_output, encoder_hidden = encoder1(
            input_tensor[ei], encoder_hidden, 32)
        encoder_outputs[ei] = encoder_output[0]
      
      decoder_hidden = encoder_hidden
      decoder_input = torch.tensor([SOS_token] * 32).to(device)

      utf = random.Random(0).random()

      if utf:
        for di in range(target_length):
          decoder_output, decoder_hidden = decoder1(decoder_input, decoder_hidden, encoder_outputs, 32, target_length)
          decoder_input = target_tensor[di]

          loss += criterion(decoder_output, target_tensor[di])

      else:
        for di in range(target_length):
          decoder_output, decoder_hidden = decoder1(decoder_input, decoder_hidden, encoder_outputs, 32, target_length)
          topv, topi = decoder_output.data.topk(1)
          decoder_input = topi.view(1, -1)[0]

          loss += criterion(decoder_output, target_tensor[di])

      loss.backward()

      encoder_optimizer.step()
      decoder_optimizer.step()

      # print(loss.item())

      epoch_loss += loss.item()/target_length

      enc_sched.step()
      dec_sched.step()

    print(epoch_loss/(len(batches)))

  torch.save(encoder1, '/content/drive/MyDrive/encoder1_attn.pt')
  torch.save(decoder1, '/content/drive/MyDrive/decoder1_attn.pt')
else:
  encoder1 = torch.load('/content/drive/MyDrive/encoder1_attn.pt', map_location = device)
  decoder1 = torch.load('/content/drive/MyDrive/decoder1_attn.pt', map_location = device)

In [ ]:
encoder1.eval()
decoder1.eval()

Attn_Decoder(
  (embedding): Embedding(2803, 256)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (rnn): GRU(256, 256, num_layers=2)
  (out): Linear(in_features=256, out_features=2803, bias=True)
)

In [ ]:
def dist_similarity_score(decoder_output, topx, model):
  topv, topi = decoder_output.data.topk(topx)
  prediction = 0

  total_score1 = 0
  for i in range(len(topi)):
    main_word = output_lang.index2word[topi[i][0].item()]
    embeddings_main_word = model.encode(main_word)
    score1 = 0
    for w1 in topi[i]:
      if topi[i][0] == w1:
        continue
      embeddings_other_word = model.encode(output_lang.index2word[w1.item()])
      score1 += util.cos_sim(embeddings_main_word, embeddings_other_word)[0][0].item()
    score1 /= len(topi[i] - 1)
    total_score1 += score1

  # for i in range(len(topi)):
  #   score2 = 0
  #   for w1 in topi[i]:
  #     for w2 in topi[i]:
  #       if w1 != w2:
  #         embedding_w1 = model.encode(output_lang.index2word[w1.item()])
  #         embedding_w2 = model.encode(output_lang.index2word[w2.item()])
  #         score2 += util.cos_sim(embedding_w1, embedding_w2)[0][0].item()
  #   score2 /= ((len(topi[i]) - 1)**2)
  #   total_score2 += score2

  return total_score1/32

In [ ]:
def new_metric(prediction, other_words, model):
  all_sentences = []
  for i in range(len(prediction[0])):
    curr_sentence = []
    for j in range(len(prediction)):
      curr_sentence.append(prediction[j][i])
    for k in range(len(curr_sentence)):
      if curr_sentence[k] == 'PAD':
        curr_sentence = curr_sentence[:k]
        break
    # all_sentences.append(' '.join(word for word in curr_sentence[1:-1]))
    all_sentences.append(curr_sentence[1:-1])

  all = []
  for i in range(len(all_sentences)):
    sen = all_sentences[i]
    # print('core sentence', sen)
    sentences = []
    for j in range(len(sen)):
      curr_other_word = other_words[j]
      all_replaced = []
      # for the curr_word stuff
      for l in range(len(curr_other_word)):
        new = sen[:]
        new[j] = output_lang.index2word[curr_other_word[l][i].item()]
        all_replaced.append(' '.join(word for word in new))
      sentences.append(all_replaced)
    all.append(sentences)

  final_score = 0
  for i in range(len(all)):
    outer_score = 0
    for j in range(len(all[i])):
      to_compare = all[i][j][0]
      # print(to_compare)
      flag_we = model.encode(to_compare)
      interscore = 0
      for k in range(1, len(all[i][j]), 1):
        other = model.encode(all[i][j][k])
        # print(util.cos_sim(flag_we, other), to_compare, all[i][j][k])
        interscore += util.cos_sim(flag_we, other)
      interscore /= (len(all[i][j]) - 1)
      outer_score += interscore
    outer_score /= (len(all[i]))
    final_score += outer_score
  final_score /= len(all)

  return final_score

In [ ]:
from tqdm.notebook import tqdm
import copy

In [ ]:
def distribution_function(test_batches, encoder, decoder):
  all_outputs_dist = []
  all_outputs = []
  with torch.no_grad():
    for sentences in tqdm(test_batches):
      all_dist = []
      input_tensor = []
      for sen in sentences[0]:
        curr_input = tensorFromSentence(input_lang, sen)
        input_tensor.append(curr_input)
      input_tensor = torch.nn.utils.rnn.pad_sequence(input_tensor)
      input_length = input_tensor.size()[0]
      encoder_hidden = encoder.init_hidden(32)
      encoder_outputs = torch.zeros(input_length, 32, encoder1.hidden_size).to(device)
      for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden, 32
        )
        encoder_outputs[ei] = encoder_output[0]
      decoder_hidden = encoder_hidden
      decoder_input = torch.tensor([SOS_token] * 32).to(device)
      output_sentence = [[SOS_token] * 32]
      for di in range(10):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs, 32, 10)
        topv, topi = decoder_output.data.topk(1)
        words_top5 = []
        topv5, topi5 = decoder_output.data.topk(5)
        for i in range(len(topi5)):
          intermediate_words = []
          for j in range(len(topi5[i])):
            intermediate_words.append(output_lang.index2word[topi5[i][j].item()])
          # print(intermediate_words)
          words_top5.append(intermediate_words)
        all_dist.append((topv5, topi5, words_top5))

        decoder_input = topi.view(1, -1)[0]
        inter = []
        for i in decoder_input:
          inter.append(output_lang.index2word[i.item()])
        output_sentence.append(inter)
        decoder_input = topi.squeeze().detach()
      all_outputs.append(output_sentence)
      all_outputs_dist.append(all_dist)
  
  return all_outputs, all_outputs_dist

In [ ]:
def produce_translations(test_batches, encoder, decoder):
  all_outputs = []
  model = SentenceTransformer('all-MiniLM-L6-v2')
  all_scores_1 = 0
  all_scores_2 = 0
  with torch.no_grad():
    for sentences in tqdm(test_batches):
      input_tensor = []
      for sen in sentences[0]:
        curr_input = tensorFromSentence(input_lang, sen)
        input_tensor.append(curr_input)
      input_tensor = torch.nn.utils.rnn.pad_sequence(input_tensor)
      input_length = input_tensor.size()[0]
      encoder_hidden = encoder.init_hidden(32)
      encoder_outputs = torch.zeros(input_length, 32, encoder1.hidden_size).to(device)
      for ei in range(input_length):
          encoder_output, encoder_hidden = encoder(
              input_tensor[ei], encoder_hidden, 32)
          encoder_outputs[ei] = encoder_output[0]
      decoder_hidden = encoder_hidden
      decoder_input = torch.tensor([SOS_token] * 32).to(device)
      output_sentence = [[SOS_token]*32]
      intermediate_score1 = 0
      all_other_words = []
      for di in range(10):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs, 32, 10)
        decoder_output_other = copy.deepcopy(decoder_output)
        topv, topi = decoder_output.data.topk(1)
        score1 = dist_similarity_score(decoder_output, 5, model)
        intermediate_score1 += score1
        decoder_input = topi.view(1, -1)[0]
        inter = []
        for i in decoder_input:
          inter.append(output_lang.index2word[i.item()])
        output_sentence.append(inter)
        decoder_input = topi.squeeze().detach()

        topv_other, topi_other = decoder_output.data.topk(5)
        # print(topi_other)
        # decoder_input_other = topi_other.view(1, 5, -1)[0]
        decoder_input_other = torch.transpose(topi_other, 0, 1)
        # print(decoder_input_other)
        all_other_words.append(decoder_input_other)
      intermediate_score1 /= 10
      all_scores_1 += intermediate_score1
      all_outputs.append(output_sentence)

      score2 = new_metric(output_sentence, all_other_words, model)
      # print(score2)
      all_scores_2 += score2
    
  return all_outputs, all_scores_1/len(test_batches), all_scores_2/len(test_batches)
  # return all_outputs

In [ ]:
def bleu_batch(test_batches, translations):
  all_predictions = []
  references = []
  for b in range(len(translations)):
    batch = translations[b]
    for i in range(len(batch[0])):
      inter = []
      for j in range(len(batch)):
        inter.append(batch[j][i])
        if batch[j][i] == '.':
          break
      references.append([test_batches[b][1][i].split(' ')])
      all_predictions.append(inter[1:])

  return corpus_bleu(references, all_predictions)

In [ ]:
test_batches = []
i = 0
while i < 2592:
  j = 0
  train_french = []
  train_english = []
  train_french_lengths = []
  train_english_lengths = []
  while j < 32:
    sf, se = test_pairs[i]
    train_french.append(sf)
    train_english.append(se)
    i += 1
    j += 1
  test_batches.append((train_french, train_english))

In [ ]:
test_batch_1 = [test_batches[0]]
print(len(test_batches), len(test_batches[0]), len(test_batches[0][0]))
print(len(test_batch_1), len(test_batch_1[0]), len(test_batch_1[0][0]))

81 2 32
1 2 32


In [ ]:
print(test_batch_1)

[(['je suis vraiment choque .', 'elle est paralysee des deux jambes .', 'je suis maintenant disposee a discuter .', 'vous n etes plus seuls .', 'je me rejouis de vous avoir finalement rencontre .', 'je suis sure que nous pouvons resoudre cela .', 'il n est pas a sa place .', 'je suis desolee . j ai oublie .', 'elles parlent .', 'je vais aller faire une sieste .', 'nous sommes trop proches .', 'c est un lambin .', 'vous me decevez beaucoup .', 'je viens de roumanie .', 'vous etes respecte de tous .', 'je suis extremement gras .', 'elle est dans une bonne passe .', 'nous allons manquer de faire ceci .', 'tu es tres contrariee .', 'vous ne visez pas assez haut .', 'vous mentez .', 'ils ne sont pas encore a la maison .', 'c est une personne de confiance .', 'elle est affreusement mauvaise en cuisine .', 'tu es nerveusement use .', 'tu es un de ces dragueurs !', 'vous etes tres contrariee .', 'nous y sommes tous ensemble .', 'vous etes toutes folles .', 'c est un mauvais conducteur .', 'nou

In [ ]:
# test_batch_1 = [(['il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .', 'il est une bonne personne .'], ['he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .', 'he is a good person .'])]

In [ ]:
translations, score1, score2 = produce_translations(test_batch_1, encoder1, decoder1)
bleu_score = bleu_batch(test_batch_1, translations)
print(bleu_score, score1, score2)

  0%|          | 0/1 [00:00<?, ?it/s]

0.38003851379374093 0.289951803595759 tensor([[0.7851]])


In [ ]:
translations, dists = distribution_function(test_batch_1, encoder1, decoder1)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dists

[[(tensor([[-1.0623e-02, -4.6822e+00, -6.6857e+00, -1.0254e+01, -1.1498e+01],
           [-3.6102e-04, -8.3508e+00, -9.2890e+00, -1.1934e+01, -1.2001e+01],
           [ 0.0000e+00, -1.8360e+01, -1.8379e+01, -1.9039e+01, -1.9188e+01],
           [-5.8888e-05, -9.7664e+00, -1.3758e+01, -1.5478e+01, -1.6130e+01],
           [-5.1855e-05, -1.0278e+01, -1.1134e+01, -1.3686e+01, -1.4142e+01],
           [-1.3406e-02, -4.3191e+00, -1.3743e+01, -1.4348e+01, -1.4613e+01],
           [-1.5544e-04, -1.0016e+01, -1.0037e+01, -1.0585e+01, -1.1088e+01],
           [ 0.0000e+00, -1.8732e+01, -1.9202e+01, -2.0235e+01, -2.1203e+01],
           [-8.5589e-05, -9.8506e+00, -1.1740e+01, -1.2043e+01, -1.2408e+01],
           [-7.5102e-06, -1.1856e+01, -1.4787e+01, -1.6576e+01, -1.8294e+01],
           [-4.1679e-04, -8.2273e+00, -9.0509e+00, -1.0492e+01, -1.3900e+01],
           [-3.5768e-04, -8.0461e+00, -1.1042e+01, -1.1192e+01, -1.3115e+01],
           [-5.0513e-01, -9.2843e-01, -6.7903e+00, -8.4665e+00, 

In [ ]:
print(translations)

[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ['he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he'], ['is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is', 'is'], ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'], ['good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good'], ['cook', 'cook', 'cook', 'cook', 'cook', 'cook', 'cook', 'cook', 'cook', 'cook', 'cook', 'co

In [ ]:
for i in range(len(translations[0][0])):
  for j in range(len(translations[0])):
    print(translations[0][j][i], end = ' ')
    if translations[0][j][i] == '.':
      break
  print('\t' + test_batch_1[0][1][i])

1 i m not a doctor . 	i m really shocked .
1 she is an excellent student . 	she is paralyzed in both legs .
1 i m ready to leave . 	i m in the mood to talk now .
1 you re not alone anymore . 	you re not alone anymore .
1 i m glad to see you again . 	i m glad to finally meet you .
1 i m sure we can work this out . 	i m sure we can work this out .
1 he s not at all busy . 	he s out of position .
1 i am sorry . 	i m sorry . i forgot .
1 they re cool . 	they re talking .
1 i m going to leave up my time . 	i m going to go take a nap .
1 we are too close . 	we re too close .
1 he s a bigot . 	he s a slowpoke .
1 i m very happy . 	i m very disappointed in you .
1 i m from kyoto . 	i m from romania .
1 you re the oldest . 	you re respected by everybody .
1 i am a stranger too . 	i m extremely fat .
1 she s a good swimmer . 	she s on a roll .
1 we re going to miss this place . 	we re going to miss doing this .
1 you re very upset . 	you re very upset .
1 you re not helping me . 	you re not aimi

In [ ]:
translations, score1, score2 = produce_translations(test_batches, encoder1, decoder1)
bleu_score = bleu_batch(test_batches, translations)
print(bleu_score, score1, score2)

  0%|          | 0/81 [00:00<?, ?it/s]

0.34763006769763466 0.29039177756451845 tensor([[0.7766]])


In [ ]:
# First Prune - 90%
hidden_size = 256
encoder1_prune3 = type(encoder1)(input_lang.n_words, hidden_size).to(device)
encoder1_prune3.load_state_dict(encoder1.state_dict())

decoder1_prune3 = type(decoder1)(hidden_size, output_lang.n_words).to(device)
decoder1_prune3.load_state_dict(decoder1.state_dict())

print('Global Sparsity for Encoder: {:.2f}%'.format(
    100. * float(
        torch.sum(encoder1_prune3.rnn.weight_ih_l0 == 0)
        + torch.sum(encoder1_prune3.rnn.weight_hh_l0 == 0)
        + torch.sum(encoder1_prune3.rnn.weight_ih_l1 == 0)
        + torch.sum(encoder1_prune3.rnn.weight_hh_l1 == 0)
    )
    / float(encoder1_prune3.rnn.weight_ih_l0.nelement()
    + encoder1_prune3.rnn.weight_hh_l0.nelement()
    + encoder1_prune3.rnn.weight_ih_l1.nelement()
    + encoder1_prune3.rnn.weight_hh_l1.nelement()
    )
)) 

print('Global Sparsity for Decoder: {:.2f}%'.format(
    100. * float(
        torch.sum(decoder1_prune3.rnn.weight_ih_l0 == 0)
        + torch.sum(decoder1_prune3.rnn.weight_hh_l0 == 0)
        + torch.sum(decoder1_prune3.rnn.weight_ih_l1 == 0)
        + torch.sum(decoder1_prune3.rnn.weight_hh_l1 == 0)
    )
    / float(decoder1_prune3.rnn.weight_ih_l0.nelement()
    + decoder1_prune3.rnn.weight_hh_l0.nelement()
    + decoder1_prune3.rnn.weight_ih_l1.nelement()
    + decoder1_prune3.rnn.weight_hh_l1.nelement()
    )
)) 

parameters_to_prune_encoder = (
    (encoder1_prune3.rnn, 'weight_ih_l0'),
    (encoder1_prune3.rnn, 'weight_hh_l0'),
    (encoder1_prune3.rnn, 'weight_ih_l1'),
    (encoder1_prune3.rnn, 'weight_hh_l1'),
)

prune.global_unstructured(
    parameters_to_prune_encoder,
    pruning_method = prune.L1Unstructured,
    amount = 0.9,
)

parameters_to_prune_decoder = (
    (decoder1_prune3.rnn, 'weight_ih_l0'),
    (decoder1_prune3.rnn, 'weight_hh_l0'),
    (decoder1_prune3.rnn, 'weight_ih_l1'),
    (decoder1_prune3.rnn, 'weight_hh_l1'),
    (decoder1_prune3.out, 'weight')
)

prune.global_unstructured(
    parameters_to_prune_decoder,
    pruning_method = prune.L1Unstructured,
    amount = 0.9,
)

print('Global Sparsity for Encoder: {:.2f}%'.format(
    100. * float(
        torch.sum(encoder1_prune3.rnn.weight_ih_l0 == 0)
        + torch.sum(encoder1_prune3.rnn.weight_hh_l0 == 0)
        + torch.sum(encoder1_prune3.rnn.weight_ih_l1 == 0)
        + torch.sum(encoder1_prune3.rnn.weight_hh_l1 == 0)
    )
    / float(encoder1_prune3.rnn.weight_ih_l0.nelement()
    + encoder1_prune3.rnn.weight_hh_l0.nelement()
    + encoder1_prune3.rnn.weight_ih_l1.nelement()
    + encoder1_prune3.rnn.weight_hh_l1.nelement()
    )
)) 

print('Global Sparsity for Decoder: {:.2f}%'.format(
    100. * float(
        torch.sum(decoder1_prune3.rnn.weight_ih_l0 == 0)
        + torch.sum(decoder1_prune3.rnn.weight_hh_l0 == 0)
        + torch.sum(decoder1_prune3.rnn.weight_ih_l1 == 0)
        + torch.sum(decoder1_prune3.rnn.weight_hh_l1 == 0)
    )
    / float(decoder1_prune3.rnn.weight_ih_l0.nelement()
    + decoder1_prune3.rnn.weight_hh_l0.nelement()
    + decoder1_prune3.rnn.weight_ih_l1.nelement()
    + decoder1_prune3.rnn.weight_hh_l1.nelement()
    )
)) 

# translations, score1, score_2 = produce_translations(test_batches, encoder1_prune3, decoder1_prune3)
# bleu_score = bleu_batch(test_batches, translations)

# print(bleu_score, score1, score2)

Global Sparsity for Encoder: 0.00%
Global Sparsity for Decoder: 0.00%
Global Sparsity for Encoder: 90.00%
Global Sparsity for Decoder: 93.72%


In [ ]:
# train this model

training_required = True

if training_required:
  epochs = 50
  criterion = nn.NLLLoss()
  encoder1_prune3.train()
  decoder1_prune3.train()
  encoder_optimizer = optim.SGD(encoder1_prune3.parameters(), lr = 0.01)
  decoder_optimizer = optim.SGD(decoder1_prune3.parameters(), lr = 0.01)

  enc_sched = torch.optim.lr_scheduler.StepLR(encoder_optimizer, 32, gamma = 0.9999)
  dec_sched = torch.optim.lr_scheduler.StepLR(decoder_optimizer, 32, gamma = 0.9999)

  teacher_forcing_ratio = 0.5

  for epoch in range(epochs):
    epoch_loss = 0
    for batch in batches:

      loss = 0 
      input_tensor = batch[0]
      target_tensor = batch[1]

      input_length = input_tensor.size()[0]
      target_length = target_tensor.size()[0]

      encoder_optimizer.zero_grad()
      decoder_optimizer.zero_grad()

      encoder_hidden = encoder1_prune3.init_hidden(32)

      encoder_outputs = torch.zeros(input_length, 32, encoder1_prune3.hidden_size).to(device)
      for ei in range(input_length):
        encoder_output, encoder_hidden = encoder1_prune3(
            input_tensor[ei], encoder_hidden, 32)
        encoder_outputs[ei] = encoder_output[0]
      
      decoder_hidden = encoder_hidden
      decoder_input = torch.tensor([SOS_token] * 32).to(device)

      utf = random.Random(0).random()

      if utf:
        for di in range(target_length):
          decoder_output, decoder_hidden = decoder1_prune3(decoder_input, decoder_hidden, encoder_outputs, 32, target_length)
          decoder_input = target_tensor[di]

          loss += criterion(decoder_output, target_tensor[di])

      else:
        for di in range(target_length):
          decoder_output, decoder_hidden = decoder1_prune3(decoder_input, decoder_hidden, encoder_outputs, 32, target_length)
          topv, topi = decoder_output.data.topk(1)
          decoder_input = topi.view(1, -1)[0]

          loss += criterion(decoder_output, target_tensor[di])

      loss.backward()

      encoder_optimizer.step()
      decoder_optimizer.step()

      # print(loss.item())

      epoch_loss += loss.item()/target_length

      enc_sched.step()
      dec_sched.step()

    print(epoch_loss/(len(batches)))

encoder1_prune3.eval()
decoder1_prune3.eval()

# translations, score1, score2 = produce_translations(test_batches, encoder1_prune3, decoder1_prune3)
# bleu_score = bleu_batch(test_batches, translations)
# print(bleu_score, score1, score2)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:951: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


1.2092390099843346
0.9363890749613442
0.8548231388409936
0.8035170892874397
0.7674471030182304
0.7391501875135632
0.7160874824788837
0.6968862758795417
0.6798342101838852
0.6653518501546649
0.6523646640300752
0.6405259619606866
0.6298707458231179
0.6198659426530204
0.6098341125806175
0.6019964654869502
0.5939298089557224
0.5863018298943833
0.5789744728194341
0.5725722268581385
0.5669581162876554
0.5603604501777222
0.5535386474609375
0.548660746547911
0.5438834368387861
0.5385924543645645
0.5328337139606476
0.5295833601209853
0.5236814458105301
0.5209283398575252
0.5153060654878615
0.5118845636420781
0.5073236649195354
0.5038439266787637
0.4996379079606796
0.4971354571130541
0.49224553288883627
0.4884408926698898
0.48555620878007655
0.4824867339981926
0.47968875876002837
0.47724006506337047
0.4742798398097358
0.4700909185727439
0.46756414706442073
0.464402687409189
0.4615884880251355
0.4588313160816833
0.456515977128347
0.4541468657784991


Attn_Decoder(
  (embedding): Embedding(2803, 256)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (rnn): GRU(256, 256, num_layers=2)
  (out): Linear(in_features=256, out_features=2803, bias=True)
)

In [ ]:
translations, dists = distribution_function(test_batch_1, encoder1_prune3, decoder1_prune3)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
translations

[[[1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  ['i',
   'she',
   'i',
   'you',
   'i',
   'i',
   'he',
   'i',
   'they',
   'i',
   'we',
   'he',
   'i',
   'i',
   'you',
   'i',
   'she',
   'we',
   'you',
   'you',
   'you',
   'they',
   'he',
   'she',
   'you',
   'you',
   'you',
   'we',
   'you',
   'he',
   'we',
   'he'],
  ['m',
   'is',
   'm',
   're',
   'm',
   'm',
   's',
   'am',
   're',
   'm',
   're',
   's',
   'm',
   'm',
   're',
   'am',
   's',
   're',
   're',
   're',
   're',
   're',
   'is',
   'is',
   're',
   're',
   're',
   're',
   're',
   'is',
   're',
   's'],
  ['really',
   'the',
   'so',
   'not',
   'glad',
   'sure',
   'not',
   'sorry',
   'awake',
   'going',
   'too',
   'a',
   'very',
   'from',
   'all',
   'a',
   'in',
   'going',
   'very',
   'not',
   'resilient',
   'not

In [ ]:
dists

[[(tensor([[-1.0736e-03, -7.1473e+00, -8.5817e+00, -9.2835e+00, -1.3406e+01],
           [-2.8118e-04, -8.8537e+00, -9.1582e+00, -1.1289e+01, -1.1791e+01],
           [-3.5763e-07, -1.5503e+01, -1.6741e+01, -1.6891e+01, -1.7374e+01],
           [-1.6226e-03, -6.4562e+00, -1.0239e+01, -1.2558e+01, -1.2684e+01],
           [-6.3537e-05, -9.9226e+00, -1.1387e+01, -1.2967e+01, -1.4811e+01],
           [-3.4968e-02, -3.3711e+00, -1.3164e+01, -1.3478e+01, -1.3550e+01],
           [-3.3659e-04, -9.1437e+00, -9.2733e+00, -1.0379e+01, -1.0424e+01],
           [-1.1921e-07, -1.6149e+01, -1.7836e+01, -1.9448e+01, -2.0979e+01],
           [-1.5675e-04, -9.9029e+00, -1.0547e+01, -1.0675e+01, -1.0941e+01],
           [-3.8147e-06, -1.2579e+01, -1.4853e+01, -1.8589e+01, -1.8989e+01],
           [-6.3149e-04, -7.5902e+00, -9.3960e+00, -1.0331e+01, -1.2843e+01],
           [-4.1393e-04, -7.8348e+00, -1.1600e+01, -1.2469e+01, -1.2929e+01],
           [-2.3398e-01, -1.5733e+00, -6.7919e+00, -9.5459e+00, 

To see if isn't just a function of extra training

In [ ]:
continue_training_required = True

hidden_size = 256
encoder1_continue = type(encoder1)(input_lang.n_words, hidden_size).to(device)
encoder1_continue.load_state_dict(encoder1.state_dict())

decoder1_continue = type(decoder1)(hidden_size, output_lang.n_words).to(device)
decoder1_continue.load_state_dict(decoder1.state_dict())

if continue_training_required:
  encoder1_continue.train()
  decoder1_continue.train()
  hidden_size = 256
  # torch.manual_seed(0)
  # torch.cuda.manual_seed(0)
  # encoder1 = encoder(input_lang.n_words, hidden_size).to(device)
  # torch.manual_seed(0)
  # torch.cuda.manual_seed(0)
  # decoder1 = Attn_Decoder(hidden_size, output_lang.n_words).to(device)
  epochs = 50
  criterion = nn.NLLLoss()
  encoder_optimizer = optim.SGD(encoder1_continue.parameters(), lr = 0.01)
  decoder_optimizer = optim.SGD(decoder1_continue.parameters(), lr = 0.01)

  enc_sched = torch.optim.lr_scheduler.StepLR(encoder_optimizer, 32, gamma = 0.9999)
  dec_sched = torch.optim.lr_scheduler.StepLR(decoder_optimizer, 32, gamma = 0.9999)
  teacher_forcing_ratio = 0.5

  for epoch in range(epochs):
    epoch_loss = 0
    for batch in batches:

      loss = 0 
      input_tensor = batch[0]
      target_tensor = batch[1]

      input_length = input_tensor.size()[0]
      target_length = target_tensor.size()[0]

      encoder_optimizer.zero_grad()
      decoder_optimizer.zero_grad()

      encoder_hidden = encoder1_continue.init_hidden(32)

      encoder_outputs = torch.zeros(input_length, 32, encoder1_continue.hidden_size).to(device)
      for ei in range(input_length):
        encoder_output, encoder_hidden = encoder1_continue(
            input_tensor[ei], encoder_hidden, 32)
        encoder_outputs[ei] = encoder_output[0]
      
      decoder_hidden = encoder_hidden
      decoder_input = torch.tensor([SOS_token] * 32).to(device)

      utf = random.Random(0).random()

      if utf:
        for di in range(target_length):
          decoder_output, decoder_hidden = decoder1_continue(decoder_input, decoder_hidden, encoder_outputs, 32, target_length)
          decoder_input = target_tensor[di]

          loss += criterion(decoder_output, target_tensor[di])

      else:
        for di in range(target_length):
          decoder_output, decoder_hidden = decoder1_continue(decoder_input, decoder_hidden, encoder_outputs, 32, target_length)
          topv, topi = decoder_output.data.topk(1)
          decoder_input = topi.view(1, -1)[0]

          loss += criterion(decoder_output, target_tensor[di])

      loss.backward()

      encoder_optimizer.step()
      decoder_optimizer.step()

      # print(loss.item())

      epoch_loss += loss.item()/target_length

      enc_sched.step()
      dec_sched.step()

    print(epoch_loss/(len(batches)))

0.2495571553177303
0.24977242472569144
0.2492960741917292
0.24588933127986062
0.2430016176276738
0.23884929943879432
0.23498759384552628
0.23077992823123922
0.22864775781896376
0.22595888349480117
0.2227012148486243
0.2211371795958941
0.21906613741715736
0.2157859440101518
0.2136419827805623
0.21166188401381164
0.2092617962254418
0.20687702268494493
0.20439329590267605
0.20085675659312127
0.19861364956961733
0.19703965880340998
0.1952219698124462
0.19301421599653007
0.19081140635013574
0.18901420423189805
0.18770840179257933
0.1858148302104737
0.18416884433693356
0.181316903187169
0.1791720668103959
0.17699700920979178
0.17469154566526404
0.1726393669366835
0.16959233737389243
0.168277169236872
0.16696774131721917
0.16501510623958388
0.162664538596736
0.1609616386956638
0.15942249877320397
0.159671262257629
0.15686297286616427
0.15482966217067518
0.1544011865933736
0.15198826750649352
0.1500779379341338
0.1486352183302244
0.14695083275503584
0.14590842544767585


In [ ]:
translations, dists = distribution_function(test_batch_1, encoder1, decoder1)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dists

[[(tensor([[-1.0449e-02, -4.6981e+00, -6.7062e+00, -1.0269e+01, -1.1513e+01],
           [-3.6042e-04, -8.3429e+00, -9.3150e+00, -1.1943e+01, -1.2005e+01],
           [ 0.0000e+00, -1.8380e+01, -1.8383e+01, -1.9044e+01, -1.9192e+01],
           [-5.8888e-05, -9.7656e+00, -1.3760e+01, -1.5474e+01, -1.6127e+01],
           [-5.1735e-05, -1.0283e+01, -1.1131e+01, -1.3682e+01, -1.4154e+01],
           [-1.3610e-02, -4.3040e+00, -1.3753e+01, -1.4348e+01, -1.4604e+01],
           [-1.5556e-04, -1.0023e+01, -1.0032e+01, -1.0579e+01, -1.1085e+01],
           [ 0.0000e+00, -1.8743e+01, -1.9205e+01, -2.0248e+01, -2.1194e+01],
           [-8.4039e-05, -9.8816e+00, -1.1729e+01, -1.2037e+01, -1.2412e+01],
           [-7.5102e-06, -1.1854e+01, -1.4787e+01, -1.6574e+01, -1.8286e+01],
           [-4.1762e-04, -8.2186e+00, -9.0648e+00, -1.0487e+01, -1.3922e+01],
           [-3.6781e-04, -8.0166e+00, -1.1041e+01, -1.1159e+01, -1.3102e+01],
           [-5.0446e-01, -9.2941e-01, -6.8019e+00, -8.4834e+00, 